In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle

from time import time
from utils.df_loader import load_adult_df
from utils.preprocessing import remove_missing_values, min_max_scale_numerical, inverse_dummy
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from alibi.explainers import CounterFactualProto, CounterFactual
from alibi_cf.utils import get_cat_vars_dict

pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False


seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88, got 80


TF version:  2.4.0-rc0
Eager execution enabled:  True


In [2]:
df, feature_names, numerical_cols, categorical_cols, columns_type, target_name, possible_outcomes = load_adult_df()

In [3]:
scaled_df, scaler = min_max_scale_numerical(df, numerical_cols)

In [4]:
scaled_df.head(5)

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,0.301370,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,0.02174,0.0,0.397959,United-States,<=50K
1,0.452055,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00000,0.0,0.122449,United-States,<=50K
2,0.287671,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.00000,0.0,0.397959,United-States,<=50K
3,0.493151,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.00000,0.0,0.397959,United-States,<=50K
4,0.150685,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.00000,0.0,0.397959,Cuba,<=50K


In [5]:
dummy_df = pd.get_dummies(scaled_df, columns=  [ col for col in categorical_cols if col != target_name])

In [6]:
### We should have this amount of input features.
sum([len(scaled_df[col].unique()) for col in categorical_cols if col != target_name]) + len(numerical_cols)

103

In [7]:
# label_enconded_df, encoder_dict = label_encode(scaled_df, categorical_cols)

In [8]:
cat_to_ohe_cat = {}
for c_col in categorical_cols:
    if c_col != target_name:
        cat_to_ohe_cat[c_col] = [ ohe_col for ohe_col in dummy_df.columns if ohe_col.startswith(c_col) and ohe_col != target_name]

In [6]:
ohe_feature_names = [ col for col in dummy_df.columns if col != target_name]

In [10]:
dummy_df.head(5)

,age,capital-gain,capital-loss,hours-per-week,class,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0.301370,0.02174,0.0,0.397959,<=50K,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.452055,0.00000,0.0,0.122449,<=50K,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.287671,0.00000,0.0,0.397959,<=50K,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,0.493151,0.00000,0.0,0.397959,<=50K,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,0.150685,0.00000,0.0,0.397959,<=50K,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# def inverse_dummy(dummy_df, cat_to_ohe_cat):
#     not_dummy_df = dummy_df.copy(deep=True)
#     for k in cat_to_ohe_cat.keys():
#         not_dummy_df[k] = dummy_df[cat_to_ohe_cat[k]].idxmax(axis=1)
#         not_dummy_df[k] = not_dummy_df[k].apply(lambda x: x.replace(f'{k}_',""))
#         not_dummy_df.drop(cat_to_ohe_cat[k], axis=1, inplace=True)
#     return not_dummy_df

In [14]:
inverse_dummy(dummy_df, cat_to_ohe_cat).head(5)

,age,capital-gain,capital-loss,hours-per-week,class,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,0.301370,0.02174,0.0,0.397959,<=50K,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,0.452055,0.00000,0.0,0.122449,<=50K,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,0.287671,0.00000,0.0,0.397959,<=50K,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,0.493151,0.00000,0.0,0.397959,<=50K,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,0.150685,0.00000,0.0,0.397959,<=50K,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba


In [13]:
# enconded_df, encoder_dict = label_encode(scaled_df, categorical_cols)

In [16]:
from sklearn.preprocessing import LabelEncoder

target_label_encoder = LabelEncoder()
dummy_df[target_name] = target_label_encoder.fit_transform(dummy_df[target_name])

In [15]:
train_df, test_df = train_test_split(dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [16]:
X_train = np.array(train_df[ohe_feature_names])
y_train = np.array(train_df[target_name])
X_test = np.array(test_df[ohe_feature_names])
y_test = np.array(test_df[target_name])

In [17]:
### Train
nn = model= tf.keras.models.Sequential(
            [
                tf.keras.layers.Dense(24,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(1),
                tf.keras.layers.Activation(tf.nn.sigmoid),
            ]
        )
nn.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['accuracy'])
nn.fit(X_train, y_train, batch_size=64, epochs=20, shuffle=True)

models = {
    "dt": DecisionTreeClassifier().fit(X_train,y_train),
    "rfc": RandomForestClassifier().fit(X_train,y_train),
    "nn": nn,
}

pickle.dump(models['dt'], open('./saved_models/dt.p', 'wb'))
pickle.dump(models['rfc'], open('./saved_models/rfc.p', 'wb'))
models['nn'].save('./saved_models/nn.h5',overwrite=True)


Epoch 1/20
407/407 [==============================] - 0s 446us/step - loss: 0.4766 - accuracy: 0.7755
Epoch 2/20
407/407 [==============================] - 0s 397us/step - loss: 0.3505 - accuracy: 0.8335
Epoch 3/20
407/407 [==============================] - 0s 378us/step - loss: 0.3362 - accuracy: 0.8426
Epoch 4/20
407/407 [==============================] - 0s 400us/step - loss: 0.3238 - accuracy: 0.8501
Epoch 5/20
407/407 [==============================] - 0s 377us/step - loss: 0.3148 - accuracy: 0.8546
Epoch 6/20
407/407 [==============================] - 0s 459us/step - loss: 0.3212 - accuracy: 0.8515
Epoch 7/20
407/407 [==============================] - 0s 391us/step - loss: 0.3105 - accuracy: 0.8558
Epoch 8/20
407/407 [==============================] - 0s 419us/step - loss: 0.3123 - accuracy: 0.8543
Epoch 9/20
407/407 [==============================] - 0s 534us/step - loss: 0.3051 - accuracy: 0.8575
Epoch 10/20
407/407 [==============================] - 0s 552us/step - loss: 0.304

In [18]:
### Load
models = {}
models['dt'] = pickle.load(open('./saved_models/dt.p', 'rb'))
models['rfc'] = pickle.load(open('./saved_models/rfc.p', 'rb'))
models['nn'] = tf.keras.models.load_model('./saved_models/nn.h5')

## Initialise NN output shape as (None, 1) for tensorflow.v1
models['nn'].predict(np.zeros((2, X_train.shape[-1])))


array([[0.21752243],
       [0.21752243]], dtype=float32)

In [19]:
example_data = X_test[0, :].reshape(1,-1)

dt_pred = models['dt'].predict(example_data)[0]
rfc_pred = models['rfc'].predict(example_data)[0]
nn_pred = models['nn'].predict(example_data)[0][0]

print(f"DT [{dt_pred}], RFC [{rfc_pred}], NN [{nn_pred}]")

DT [0], RFC [0], NN [0.46238642930984497]


# DiCE

In [20]:
import dice_ml

In [21]:
scaled_df

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,0.301370,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,0.021740,0.0,0.397959,United-States,<=50K
1,0.452055,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.000000,0.0,0.122449,United-States,<=50K
2,0.287671,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.000000,0.0,0.397959,United-States,<=50K
3,0.493151,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.000000,0.0,0.397959,United-States,<=50K
4,0.150685,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.000000,0.0,0.397959,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.136986,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,0.000000,0.0,0.377551,United-States,<=50K
32557,0.315068,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.000000,0.0,0.397959,United-States,>50K
32558,0.561644,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,0.000000,0.0,0.397959,United-States,<=50K
32559,0.068493,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,0.000000,0.0,0.193878,United-States,<=50K


In [22]:
d = dice_ml.Data(dataframe=scaled_df, continuous_features=numerical_cols, outcome_name=target_name)

In [23]:
# m = dice_ml.Model(model=models['dt'], backend="sklearn")
# exp = dice_ml.Dice(d,m)

In [24]:
class RecordWrapper():
    def __init__(self, model, cat_to_ohe_cat, ohe_feature_names):
        self.all_inputs = []
        self.model = model
        self.cat_to_ohe_cat = cat_to_ohe_cat
        self.ohe_feature_names = ohe_feature_names


    def dice_to_input(self, input_df):
        x = input_df.copy(deep=True)

        for k in cat_to_ohe_cat.keys():
            for ohe_col in cat_to_ohe_cat[k]:
                x[ohe_col] = x[k].apply(lambda v: 1 if v in ohe_col else 0) 
            x.drop([k], axis=1, inplace=True)
            
        return np.array(x[ohe_feature_names])

    def predict_proba(self, x):
        self.all_inputs.append(x)
        cf_input = self.dice_to_input(x)
        return self.model.predict_proba(cf_input)

    def predict(self, x):
        self.all_inputs.append(x)
        cf_input = self.dice_to_input(x)
        return self.model.predict(cf_input)

class NNRecordWrapper():
    def __init__(self, model, cat_to_ohe_cat, ohe_feature_names):
        self.all_inputs = []
        self.model = model
        self.cat_to_ohe_cat = cat_to_ohe_cat
        self.ohe_feature_names = ohe_feature_names


    def dice_to_input(self, input_df):
        x = input_df.copy(deep=True)

        for k in cat_to_ohe_cat.keys():
            for ohe_col in cat_to_ohe_cat[k]:
                x[ohe_col] = x[k].apply(lambda v: 1 if v in ohe_col else 0) 
            x.drop([k], axis=1, inplace=True)
            
        return np.array(x[ohe_feature_names])
    
    def predict(self, x):
        self.all_inputs.append(x)
        cf_input = self.dice_to_input(x)
        return self.model.predict(tf.constant(cf_input.astype(float)))

    def predict_proba(self, x):
        self.all_inputs.append(x)
        cf_input = self.dice_to_input(x)
        return self.model.predict(tf.constant(cf_input.astype(float)))
        

In [25]:
all_wrapped_models = {
    'dt': RecordWrapper(models['dt'], cat_to_ohe_cat, ohe_feature_names),
    'rfc': RecordWrapper(models['rfc'], cat_to_ohe_cat, ohe_feature_names),
    'nn': NNRecordWrapper(models['nn'], cat_to_ohe_cat, ohe_feature_names),
}

In [26]:
# dt_record = RecordWrapper(models['dt'], cat_to_ohe_cat, ohe_feature_names)
# m = dice_ml.Model(model=dt_record, backend="sklearn")
# exp = dice_ml.Dice(d,m)
# dice_exp = exp.generate_counterfactuals(scaled_df.iloc[1:2], total_CFs=5, desired_class="opposite")
# dice_exp.cf_examples_list[0].final_cfs_df.iloc[0][:-1]

In [27]:
dice_cfs = {
    'dt': dice_ml.Dice(d,dice_ml.Model(model=all_wrapped_models['dt'], backend="sklearn")),
    'rfc': dice_ml.Dice(d,dice_ml.Model(model=all_wrapped_models['rfc'], backend="sklearn")),
    'nn': dice_ml.Dice(d,dice_ml.Model(model=all_wrapped_models['nn'], backend="sklearn"))
}

In [28]:
num_instances = 5
num_cf_per_instance = 1

In [2]:
results = {}

for k in dice_cfs.keys():
    results[k] = []
    print(f"Finding counterfactual for {k}")
    for idx, instance in enumerate(scaled_df.iloc[test_df[0:num_instances].index].iloc):
        print(f"instance {idx}")
        for num_cf in range(num_cf_per_instance):
            print(f"CF {num_cf}")
            start_t = time()

            input_query = pd.DataFrame([instance.to_dict()])
            ground_truth = input_query[target_name][0]
            exp = dice_cfs[k].generate_counterfactuals(input_query, total_CFs=1, sample_size=200, desired_class="opposite")

            # dice_exp = dice_cfs['nn'].generate_counterfactuals(scaled_df.iloc[1:2], total_CFs=1, desired_class="opposite")
            # dice_exp.cf_examples_list[0].final_cfs_df.iloc[0][:-1]

            if k=='nn':
                prediction = target_label_encoder.inverse_transform((all_wrapped_models[k].predict(input_query)[0]> 0.5).astype(int))[0]
            else:
                prediction = target_label_encoder.inverse_transform(all_wrapped_models[k].predict(input_query))[0]
            
            end_t = time ()
            running_time = end_t - start_t
            results[k].append({
                "input": input_query,
                "cf": exp.cf_examples_list[0].final_cfs_df,
                "running_time": running_time,
                "ground_truth": ground_truth,
                "prediction": prediction,
            })

NameError: name 'dice_cfs' is not defined

In [30]:
# scaled_input_df = results['dt'][0]['input'].copy(deep=True)
# origin_columns = [f"origin_input_{col}"  for col in scaled_input_df.columns]
# origin_input_df = scaled_input_df.copy(deep=True)
# scaled_input_df.columns = [f"scaled_input_{col}"  for col in scaled_input_df.columns]

# origin_input_df[numerical_cols] = scaler.inverse_transform(origin_input_df[numerical_cols])
# origin_input_df.columns = origin_columns

# scaled_cf_df = results['dt'][0]['cf'].copy(deep=True)
# scaled_cf_df.loc[0, target_name] = target_label_encoder.inverse_transform([scaled_cf_df.loc[0, target_name]])[0]
# origin_cf_columns = [f"origin_cf_{col}"  for col in scaled_cf_df.columns]
# origin_cf_df = scaled_cf_df.copy(deep=True)
# scaled_cf_df.columns = [f"scaled_cf_{col}"  for col in scaled_cf_df.columns]

# origin_cf_df[numerical_cols] = scaler.inverse_transform(origin_cf_df[numerical_cols])
# origin_cf_df.columns = origin_cf_columns

# final_df = pd.DataFrame([{}])
# final_df = final_df.join([scaled_input_df, origin_input_df, scaled_cf_df, origin_cf_df])
# final_df['running_time'] = results['dt'][0]['running_time']
# final_df['Found'] = "Y" if not results['dt'][0]['cf'] is None else "N"


In [1]:
all_df = {}

for k in results.keys():

    all_data = []

    for i in range(len(results[k])):
        final_df = pd.DataFrame([{}])

        scaled_input_df = results[k][i]['input'].copy(deep=True)
        origin_columns = [f"origin_input_{col}"  for col in scaled_input_df.columns]
        origin_input_df = scaled_input_df.copy(deep=True)
        scaled_input_df.columns = [f"scaled_input_{col}"  for col in scaled_input_df.columns]

        origin_input_df[numerical_cols] = scaler.inverse_transform(origin_input_df[numerical_cols])
        origin_input_df.columns = origin_columns

        final_df = final_df.join([scaled_input_df, origin_input_df])

        if not results[k][i]['cf'] is None:
            scaled_cf_df = results[k][i]['cf'].copy(deep=True)
            scaled_cf_df.loc[0, target_name] = target_label_encoder.inverse_transform([scaled_cf_df.loc[0, target_name]])[0]
            origin_cf_columns = [f"origin_cf_{col}"  for col in scaled_cf_df.columns]
            origin_cf_df = scaled_cf_df.copy(deep=True)
            scaled_cf_df.columns = [f"scaled_cf_{col}"  for col in scaled_cf_df.columns]

            origin_cf_df[numerical_cols] = scaler.inverse_transform(origin_cf_df[numerical_cols])
            origin_cf_df.columns = origin_cf_columns

            final_df = final_df.join([scaled_cf_df, origin_cf_df])

        # final_df = final_df.join([scaled_input_df, origin_input_df, scaled_cf_df, origin_cf_df])
        final_df['running_time'] = results[k][i]['running_time']
        final_df['Found'] = "Y" if not results[k][i]['cf'] is None else "N"
        final_df['ground_truth'] = results[k][i]['ground_truth'] 
        final_df['prediction'] = results[k][i]['prediction'] 

        all_data.append(final_df)

    all_df[k] = pd.concat(all_data)

NameError: name 'results' is not defined

In [32]:
for df_k in all_df.keys():
    all_df[df_k].to_csv(f"./results/dice_adult_{df_k}_result.csv")